# Wyjaśnienie jak wyodrębniłam dane

### Wgranie bibliotek i danych

In [2]:
#Biblioteki
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime

#Zmienienie formatu zapisu danych numerycznych na dwie cyfry po przecinku.
pd.options.display.float_format = '{:.2f}'.format

#Wczytanie danych
df = pd.read_csv('all_energy_statistics.csv')

### Podział kolumny 'commodity_transaction'

źródło kodu: https://www.kaggle.com/code/gabrielapiwar/cleaning-of-commodities-column/edit

In [3]:
#Podział na trzy kolumny
split_commodities = df.commodity_transaction.str.split(" - | – ",  expand=True)

#Przypisanie nazw kolumn
split_commodities.columns = ["commodity", "transaction_type", "additional_transaction_info"]

#Oczyszczenie stringów w kolumnie transaction_type
split_commodities.transaction_type = split_commodities.transaction_type.str.lower().str.strip()

#Poprawienie błędów w pisowni i formatowaniu
split_commodities.transaction_type = split_commodities.transaction_type.str.replace("transformatin", "transformation")
split_commodities.transaction_type = split_commodities.transaction_type.str.replace("non energy uses", "consumption for non-energy uses")
split_commodities.transaction_type = split_commodities.transaction_type.str.replace(" /", "/")
split_commodities.transaction_type = split_commodities.transaction_type.str.replace("/ ", "/")

#Dodanie trzech nowych kolumn do orginalnych danych
new_df = df.join(split_commodities)

#Usunięcie kolumny 'commodity_transaction'
new_df = new_df.drop(columns = ['commodity_transaction'])

#Zapisanie nowego pliku z danymi
with open("commodity_split_data.csv", "w+") as file:
    file.write(new_df.to_csv())

In [28]:
new_df['category'].unique()

array(['additives_and_oxygenates', 'animal_waste', 'anthracite',
       'aviation_gasoline', 'bagasse', 'biodiesel', 'biogases',
       'biogasoline', 'bitumen', 'black_liquor', 'blast_furnace_gas',
       'brown_coal_briquettes', 'brown_coal', 'charcoal', 'coal_tar',
       'coke_oven_coke', 'coking_coal', 'conventional_crude_oil',
       'direct_use_of_geothermal_heat',
       'direct_use_of_solar_thermal_heat',
       'electricity_net_installed_capacity_of_electric_power_plants',
       'ethane', 'falling_water', 'fuel_oil', 'fuelwood', 'gas_coke',
       'gas_oil_diesel_oil', 'gasoline_type_jet_fuel', 'gasworks_gas',
       'geothermal', 'hard_coal', 'heat', 'hydro', 'industrial_waste',
       'kerosene_type_jet_fuel', 'lignite', 'liquified_petroleum_gas',
       'lubricants', 'motor_gasoline', 'municipal_wastes', 'naphtha',
       'natural_gas_including_lng', 'natural_gas_liquids',
       'nuclear_electricity', 'of_which_biodiesel',
       'of_which_biogasoline', 'oil_shale_oil_sa

### Podgląd unikatowych wartości dla każdej kolumny

In [4]:
#Stworzenie słownika z unikatowymi wartościami w kolumnach 
unique_column_values = {}
keys = list(new_df.columns)
for col in keys:
    unique_column_values[col] = new_df[col].unique() 

#Posortowanie kolumny year żeby było czytelniej
unique_column_values['year'].sort()
    
#wypisanie nazw kolumn   
print(unique_column_values.keys(),"\n")

#print("Unikatowe wartości w kolumnie 'country_or_area':\n{}\n".format(unique_column_values['country_or_area']))
print("Unikatowe wartości w kolumnie 'year':\n{}\n".format(unique_column_values['year']))
print("Unikatowe wartości w kolumnie 'unit':\n{}\n".format(unique_column_values['unit']))
print("Unikatowe wartości w kolumnie 'category':\n{}\n".format(unique_column_values['category']))
print("Unikatowe wartości w kolumnie 'commodity':\n{}\n".format(unique_column_values['commodity']))
print("Unikatowe wartości w kolumnie 'transaction_type':\n{}\n".format(unique_column_values['transaction_type']))
print("Unikatowe wartości w kolumnie 'additional_transaction_info':\n{}\n".format(unique_column_values['additional_transaction_info']))

dict_keys(['country_or_area', 'year', 'unit', 'quantity', 'quantity_footnotes', 'category', 'commodity', 'transaction_type', 'additional_transaction_info']) 

Unikatowe wartości w kolumnie 'year':
[1990 1991 1992 1993 1994 1995 1996 1997 1998 1999 2000 2001 2002 2003
 2004 2005 2006 2007 2008 2009 2010 2011 2012 2013 2014]

Unikatowe wartości w kolumnie 'unit':
['Metric tons,  thousand' 'Terajoules' 'Kilowatts,  thousand'
 'Kilowatt-hours, million' 'Cubic metres, thousand' 'Metric Tons']

Unikatowe wartości w kolumnie 'category':
['additives_and_oxygenates' 'animal_waste' 'anthracite'
 'aviation_gasoline' 'bagasse' 'biodiesel' 'biogases' 'biogasoline'
 'bitumen' 'black_liquor' 'blast_furnace_gas' 'brown_coal_briquettes'
 'brown_coal' 'charcoal' 'coal_tar' 'coke_oven_coke' 'coking_coal'
 'conventional_crude_oil' 'direct_use_of_geothermal_heat'
 'direct_use_of_solar_thermal_heat'
 'electricity_net_installed_capacity_of_electric_power_plants' 'ethane'
 'falling_water' 'fuel_oil' 'fuelwood

### Wyodrębnienie elektryczności z OZE

Zanim przejdę do działania objaśnię mój tok myślowy jak podzieliłam wszystko. Na początek sprawdziłam jednostki 

In [68]:
#Pracujemy tylko na jednym commodity (dane się powtarzają między różnymi commodity, aby uprościć wybieramy tylko jedną kategorię)
eco_df = new_df.loc[new_df['commodity'] == 'Electricity']

In [69]:
#Wypisanie transaction type które nas interesuje z Electricity
eco_transaction_type = ['total geothermal production', 'total hydro production', 'total solar production', 
                        'total tide, wave production', 'total wind production', 'total production, main activity', 
                        'gross production', 'gross demand','total production, autoproducer', 'imports', 'exports']

In [70]:
#Stworzenie nowego datasetu tylko z wybranymi transaction type
eco_df = eco_df.loc[eco_df['transaction_type'].isin(eco_transaction_type)]

In [116]:
#Podział na regiony/kraje które nas interesują
EU_country = ["Austria", "Belgium", "Bulgaria", "Croatia", "Cyprus", "Czech Republic", "Denmark", "Estonia",
              "Finland", "France", "Greece", "Spain", "Netherlands", "Ireland", "Lithuania", "Luxembourg",
              "Latvia", "Malta", "Germany", "Poland", "Portugal", "Romania", "Slovakia", "Slovenia", "Sweden", 
              "Hungary", "Italy", "United Kingdom"]

World = ["United States", "China", "India", "Russian Federation"]

In [78]:
#Stworzenie nowego datasetu tylko z wybranymi regionami/krajami
EU_world_df = eco_df.loc[eco_df['country_or_area'].isin(UE_country + World)]
EU_world_df

,country_or_area,year,unit,quantity,quantity_footnotes,category,commodity,transaction_type,additional_transaction_info
490921,Austria,2014,"Kilowatt-hours, million",0.00,NaN,geothermal,Electricity,total geothermal production,None
490922,Austria,2013,"Kilowatt-hours, million",0.00,NaN,geothermal,Electricity,total geothermal production,None
490923,Austria,2012,"Kilowatt-hours, million",1.00,NaN,geothermal,Electricity,total geothermal production,None
490924,Austria,2011,"Kilowatt-hours, million",1.00,NaN,geothermal,Electricity,total geothermal production,None
490925,Austria,2010,"Kilowatt-hours, million",1.00,NaN,geothermal,Electricity,total geothermal production,None
...,...,...,...,...,...,...,...,...,...
1189456,United States,1994,"Kilowatt-hours, million",3483.00,NaN,wind_electricity,Electricity,total wind production,None
1189457,United States,1993,"Kilowatt-hours, million",3053.00,NaN,wind_electricity,Electricity,total wind production,None
1189458,United States,1992,"Kilowatt-hours, million",2917.00,NaN,wind_electricity,Electricity,total wind production,None
1189459,United States,1991,"Kilowatt-hours, million",3051.00,NaN,wind_electricity,Electricity,total wind production,None


In [77]:
#sprawdzenie jednostek - dane tylko w kilowatach na godzinę, czyli jednostce energii elektrycznej
EU_world_df['unit'].unique()

array(['Kilowatt-hours, million'], dtype=object)

In [44]:
#sprawdzenie category - 
EU_world_df['category'].unique()

array(['geothermal', 'hydro', 'solar_electricity',
       'tide_wave_and_ocean_electricity', 'total_electricity',
       'wind_electricity'], dtype=object)

In [172]:
#suma energii wyprodukowanej w danym kraju, z podziałem na jej rodzaje (suma dla wszystkich lat lub wybrany rok)

#Wybranie rocznika (można pominąć lub wybrać np. kilka lat)
#EU_world_df = EU_world_df.loc[EU_world_df['year'] == 2014]

#sumowanie energii osobno dla każdego kraju dla wszystkich roczników
sum_energy_df = EU_world_df[['country_or_area', 'quantity', 'transaction_type']].pivot_table(index='country_or_area',
                                                                                columns='transaction_type',
                                                                               aggfunc = np.sum)
#Podgląd danych
#sum_energy_df

quantity                                           \
transaction_type      exports gross demand gross production    imports   
country_or_area                                                          
Austria             335813.00   1457467.00       1529690.00  390669.00   
Belgium             186606.00   2027313.00       2033493.00  308631.00   
Bulgaria            154439.00    847937.00       1068924.00   50859.00   
China               252116.00  54131715.00      58804770.60   87305.00   
Croatia              75306.00    352818.00        276381.00  163821.00   
Cyprus                    NaN     87886.00         92481.00        NaN   
Denmark             243075.00    864930.00        948103.00  209373.00   
Estonia              57110.00    172345.00        234777.00   20539.00   
Finland              52419.00   1962318.00       1776119.00  317815.00   
France             1633762.00  11102400.00      13289334.00  187359.00   
Germany            1183777.00  12887914.00      14146393.00 1013711.00   
Greece               42821.00   1236031.00       1285639.00  106231.00   
Hungary             124164.00    933444.00        866852.00  256373.00   
India                 4147.00  15603930.00      16539859.00   73831.00   
Ireland               3405.00    557138.00        580659.00   18791.00   
Italy                33423.00   7431370.00       6837341.00 1096071.00   
Latvia               28057.00    152177.00        110412.00   76303.00   
Lithuania           143389.00    226950.00        297939.00  115123.00   
Luxembourg           44394.00    144000.00         64395.00  153692.00   
Malta                     NaN     45687.00         48521.00        NaN   
Netherlands         138722.00   2596803.00       2355011.00  477325.00   
Poland              266603.00   3224791.00       3715602.00  155508.00   
Portugal             73061.00   1081726.00       1060337.00  143246.00   
Romania              69727.00   1311915.00       1455755.00   56628.00   
Russian Federation  580975.00  19907649.00      21847563.00  219004.00   
Slovakia            174927.00    585311.00        650934.00  177716.00   
Slovenia            136368.00    279259.00        334952.00  104844.00   
Spain               236253.00   5556765.00       5931164.00  196669.00   
Sweden              377042.00   3499543.00       3731851.00  320874.00   
United Kingdom       35678.00   8854050.00       9089874.00  324200.00   
United States       377445.00  93789000.00      98746617.00 1129039.00   

                                                                       \
transaction_type   total geothermal production total hydro production   
country_or_area                                                         
Austria                                  22.00              986426.00   
Belgium                                    NaN               36094.00   
Bulgaria                                   NaN               81956.00   
China                                      NaN             9912320.00   
Croatia                                    NaN              149325.00   
Cyprus                                     NaN                    NaN   
Denmark                                    NaN                 608.00   
Estonia                                    NaN                 342.00   
Finland                                    NaN              330300.00   
France                                     NaN             1679584.00   
Germany                                 287.00              599665.00   
Greece                                     NaN              107450.00   
Hungary                                    NaN                4890.00   
India                                      NaN             2280019.00   
Ireland                                    NaN               25717.00   
Italy                                116798.00             1186077.00   
Latvia                                     NaN               67357.00   
Lithuania                                  N

In [174]:
#wartości Nan zamieniamy na 0.0 i usuwamy MultiIndex'owanie
sum_energy_df.fillna(0, inplace=True)
sum_energy_df.columns = sum_energy_df.columns.droplevel()
sum_energy_df.columns.name = None
sum_energy_df.reset_index(inplace=True)

#Podgląd danych
#sum_energy_df

In [186]:
#sumowanie energii z krajów europejskich - stworzenie jednego wiersza EU countries
eu_energy_df = sum_energy_df.loc[sum_energy_df['country_or_area'].isin(EU_country)].sum(numeric_only = True)
eu_energy_df = eu_energy_df.to_frame().transpose()
eu_energy_df['country_or_area'] = 'EU countries'
eu_energy_df

,exports,gross demand,gross production,imports,total geothermal production,total hydro production,"total production, autoproducer","total production, main activity",total solar production,"total tide, wave production",total wind production,country_or_area
0,5850341.00,69480288.00,73812933.00,6442371.00,119581.00,8922281.00,5861643.00,67951290.00,347473.00,12167.00,1771314.00,EU countries


In [188]:
#Połączenie EU i World
world_energy_df = sum_energy_df.loc[sum_energy_df['country_or_area'].isin(World)] 

final_energy_df = pd.concat([world_df, eu_energy_df])
final_energy_df.reset_index(inplace=True, drop = True)

#Podgląd danych
final_energy_df

,country_or_area,exports,gross demand,gross production,imports,total geothermal production,total hydro production,"total production, autoproducer","total production, main activity",total solar production,"total tide, wave production",total wind production
0,China,252116.00,54131715.00,58804770.60,87305.00,0.00,9912320.00,3016994.00,55787776.60,20753.00,0.00,508206.00
1,India,4147.00,15603930.00,16539859.00,73831.00,0.00,2280019.00,1868703.00,14671156.00,8453.00,0.00,208900.00
2,Russian Federation,580975.00,19907649.00,21847563.00,219004.00,5955.00,3907008.00,1240599.00,20606964.00,160.00,0.00,172.00
3,United States,377445.00,93789000.00,98746617.00,1129039.00,410355.00,7520138.00,5787864.00,92958753.00,81665.00,0.00,1002367.00
4,EU countries,5850341.00,69480288.00,73812933.00,6442371.00,119581.00,8922281.00,5861643.00,67951290.00,347473.00,12167.00,1771314.00


In [189]:
#dodanie kolumny z sumą całej energi z OZE na kraj
sum_electricity_list = ['total geothermal production', 'total hydro production',
                        'total solar production', 'total tide, wave production']

#Skrót RE oznacza Renewable Energy
final_energy_df['total RE production'] = final_energy_df[sum_electricity_list].sum(axis = 1)

#Podgląd danych
#final_energy_df

In [190]:
#dodanie kolumny z procentem jaki stanowi energia odnawialna w produkcji elektrycznosci
final_energy_df['RE production %'] = (final_energy_df['total RE production'] / final_energy_df['gross production'] * 100)

#dodanie kolumny ile % stanowi produkowana elektryczność w stosunku do zapotrzebowania
final_energy_df['RE production to demand %'] = (final_energy_df['total RE production'] / final_energy_df['gross demand'] * 100)

#dodanie kolumny z różnicą w procentach
final_energy_df['sub'] = final_energy_df['RE production to demand %'] - final_energy_df['RE production %']

#Podgląd danych
#final_energy_df

In [191]:
#Ostateczna wersja tabeli - podgląd danych
final_energy_df

,country_or_area,exports,gross demand,gross production,imports,total geothermal production,total hydro production,"total production, autoproducer","total production, main activity",total solar production,"total tide, wave production",total wind production,total RE production,RE production %,RE production to demand %,sub
0,China,252116.00,54131715.00,58804770.60,87305.00,0.00,9912320.00,3016994.00,55787776.60,20753.00,0.00,508206.00,9933073.00,16.89,18.35,1.46
1,India,4147.00,15603930.00,16539859.00,73831.00,0.00,2280019.00,1868703.00,14671156.00,8453.00,0.00,208900.00,2288472.00,13.84,14.67,0.83
2,Russian Federation,580975.00,19907649.00,21847563.00,219004.00,5955.00,3907008.00,1240599.00,20606964.00,160.00,0.00,172.00,3913123.00,17.91,19.66,1.75
3,United States,377445.00,93789000.00,98746617.00,1129039.00,410355.00,7520138.00,5787864.00,92958753.00,81665.00,0.00,1002367.00,8012158.00,8.11,8.54,0.43
4,EU countries,5850341.00,69480288.00,73812933.00,6442371.00,119581.00,8922281.00,5861643.00,67951290.00,347473.00,12167.00,1771314.00,9401502.00,12.74,13.53,0.79


In [192]:
#Zapisanie tabeli do pliku csv
with open("sum_eco_energy_data_eu_world.csv", "w+") as file:
    file.write(final_energy_df.to_csv())

### Wyodrębnienie energii z biomasy

In [16]:
#biomass_commodity_list = ['Bagasse', 'Biodiesel', 'Biogases', 'Biogasoline', 'Other liquid biofuels', 'Peat (for fuel use)']
biomass_commodity_list = ['Biodiesel']